<h1><center>Laboratorio 6: El Pandas no Muerde (act IV) 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Pablo Badilla, Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias, Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `http://....`


## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar los paradigmas y buenas prácticas de programación vistas hasta este momento.
- Comprender y aprovechar las ventajas que nos ofrece la liberia `numpy` con respecto a trabajar en Python 'puro'.
- Visualizar aplicaciones de filtros de imágenes sin el uso de librerías.
- Verificar que el uso indiscriminado de `for` puede afectar en la eficiencia en al procesar datos masivos.


El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
!pip install --upgrade pandas
!pip install xlrd
!pip install openpyxl
import pandas as pd
import datetime
from scipy import stats

from IPython.display import display, Markdown, Latex

#Libreria para plotear
!pip install --upgrade plotly
!pip install missingno
import matplotlib.pyplot as plt
import missingno as msno
import plotly.express as px

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin Mora quien es el gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas.

Para esto, el área de ventas les entrega el archivo `online_retail_II.pickle` con el que se les pide que cargue y visualicen algunas de las filas que componen el Dataset.

Realice una primera visualización de los datos y señale los atributos que componen el dataset. Señale las columnas que conforman el dataset, el tipo de variable presente en cada columna y comente que representa cada una de estas.

**Respuesta:**

In [ ]:
df_retail = pd.read_pickle("online_retail_II.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": str,
        "Customer ID": "category",
    }
)
df_retail.head()

## 1.2 Analisís Explotatorio de los Datos [0.5 puntos] 

En base a la primera visualización del dataset, *Don Mora* le solicita que realicen un análisis exploratorio de los datos, para esto les deberán realizar un análisis univariado y multivariado. De la revisión, ustedes deben explicar potenciales anomalías visualizadas y señalar si existe la necesidad de realizar una limpieza de datos.

Explique a que nos referimos con análisis univariable, multivariable y de datos faltantes. ¿Qué beneficios nos otorga estudiar estos datos?. Sea conciso con su respuesta y no escriba mas de 5 líneas para su respuesta.

**Respuesta a la Pregunta:**

> El análisis univariable se enfoca en una variable a la vez, revelando su distribución y características. El análisis multivariable examina múltiples variables simultáneamente, revelando sus relaciones mutuas. En el análisis univariable se emplean medidas de tendencia central y dispersión, mientras que el análisis multivariable utiliza técnicas como correlación y regresión. Estas funciones son esenciales para obtener información precisa, identificar patrones y tomar decisiones fundamentadas en ciencia de datos.

### 1.2.1 Análisis Univariado [2 Puntos]

A continuación, se le presentan dos funciones para analizar los datos que componen un dataframe. La primera de estas es la función ``profile_serie()`` la cual recibe una serie y le entrega un análisis detallado de los datos que conforman dicha serie. 

Ejecute la funcion ``profile_serie()`` sobre cada serie para realizar un análisis univariado de estas. A continuación, comente acerca de el comportamiento de cada variable según las estadísticas descriptivas y los gráficos generados.

In [ ]:
from pandas.api.types import is_numeric_dtype
from pandas.core.dtypes.common import is_datetime_or_timedelta_dtype


def profile_serie(serie_in, n_samples=1000, random_state=42):
    serie = serie_in.copy()

    profile = pd.Series(dtype='object')
    profile["Type"] = serie.dtype
    profile = pd.concat([profile, serie.describe()])

    # profile = pd.Series([])

    if is_numeric_dtype(serie):
        profile["Negative"] = (serie < 0).sum()
        profile["Negative (%)"] = (
            str(round((serie < 0).sum() / len(serie) * 100, 2)) + " %"
        )
        profile["Zeros"] = (serie == 0).sum()
        profile["Zeros (%)"] = (
            str(round((serie == 0).sum() / len(serie) * 100, 2)) + " %"
        )
        profile["Kurt"] = serie.kurt()
        profile["Skew"] = serie.skew()

    profile[" "] = " "  # espacio

    profile["Missing cells"] = serie.isnull().sum()
    profile["Missing cells (%)"] = (
        str(round(serie.isnull().sum() / len(serie) * 100, 2)) + " %"
    )
    profile["Duplicate rows"] = serie.duplicated(False).sum()
    profile["Duplicate rows (%)"] = (
        str(round(serie.duplicated(False).sum() / len(serie) * 100, 2)) + " %"
    )
    profile["Total size in memory"] = str(serie.memory_usage(index=True)) + " bytes"

    # profile = pd.concat([profile, description])

    profile = profile.rename(
        index={
            "count": "Number of observations",
            "mean": "Mean",
            "std": "Std",
            "min": "Min",
            "max": "Max",
            "unique": "Unique",
            "top": "Top",
            "freq": "Freq",
        }
    )
    no_outliers_fig = None

    if is_numeric_dtype(serie):

        sampled_serie = serie.sample(n_samples, random_state=random_state)
        fig = px.histogram(
            sampled_serie, marginal="box", title=f"{serie.name} - With Outliers"
        )

        no_outliers = sampled_serie.loc[(np.abs(stats.zscore(sampled_serie)) < 3)]
        # zscore = https://es.wikipedia.org/wiki/Unidad_tipificada
        
        no_outliers_fig = px.histogram(
            no_outliers, marginal="box", title=f"{serie.name} - Without Outliers"
        )

    elif is_datetime_or_timedelta_dtype(serie):
        sampled_serie = serie.sample(n_samples, random_state=random_state)
        fig = px.histogram(sampled_serie, marginal="box", title=f"{serie.name}")
    
    else:
        count = (
            serie.value_counts()[0:100]
            .reset_index()
            .rename(columns = {'count': 'Count'})
        )
        fig = px.bar(
            x=count[serie.name].astype(str),
            y=count["Count"],
            title=f"100 Most common categories of {serie.name}",
        )
    display(Markdown(f'## {serie.name} Profile'))
    display(profile)
    fig.show()

    if no_outliers_fig:
        no_outliers_fig.show()

    # return fig, profile

In [ ]:
df_retail_2  = df_retail[df_retail['Price']  == 0 ]
df_retail_2

In [ ]:
df_retail_2[df_retail_2['Quantity'] >= 0]

In [ ]:
profile_serie(df_retail['Price']) 




La variable "Price" tiene un rango amplio, desde un valor mínimo de -53.594,36 hasta un valor máximo de 25.111,09. Esto sugiere una gran variabilidad en los precios observados en el dataset.

La media de la variable es de aproximadamente 4,69 y la desviación estándar es alta, con un valor de alrededor de 146,13. Si bién se tiene esta esta desviación en el gráfico se puede notar que gran parte de los datos se encuentran concentrados cerca de la mediana, la cual es 2,1. Esto sumado a que los cuartiles 1 y 3 se encuentran entre 1,25 y 4,1 se puede notar que existen demasiados outliers con valores demasiado altos, ya que el promedio se encuentra fuera de estos percentiles.

Los valores negativos se presentan en 3 observaciones. Esto para la variable de precio no hace sentido, lo que significa que existe un error en el valor. Esto entendiendo que se encuentran en un contexto de venta y no de deuda.

Hay 3,687 observaciones con valores iguales a cero, lo que también es malo en el contexto de los precios. LAs posibilidades donde puede ocurrir esto, se regalaron los productos, u ocurrió un problema de traspaso de la información.

El coeficiente de asimetría es negativo y tiene un valor de aproximadamente -140,77. Esto indica una asimetría negativa extremadamente alta, lo que sugiere una distribución muy sesgada hacia la izquierda.

El coeficiente de curtosis es extremadamente alto, con un valor de aproximadamente 64.868,34. Esto indica una distribución muy leptocúrtica, con colas pesadas y una concentración de valores alrededor de la media.


In [ ]:
profile_serie(df_retail['Invoice']) 

In [ ]:
df_retail[df_retail['Quantity'] == 0]

La variable "Invoice" es de tipo categórico y contiene 525.461 observaciones en total.

Hay 28,816 valores únicos en la variable "Invoice", lo que indica una amplia variedad de facturas diferentes en el dataset.

El valor más frecuente en la variable "Invoice" es "537434", que aparece 675 veces en el dataset. Como es posible que distintos tipos de clientes tengan una facturación similar es posible que se tengan una facturación parecida. En particular se tiene que el 98,48% de los Invoice están repetidos. Si bien algunos puedden estar repetidos por errores en el dataset, es un número plausble.


In [ ]:
profile_serie(df_retail['StockCode']) 

La variable "StockCode" es de tipo categórico y contiene 525.461 observaciones en total. En los que existen 4.632 valores únicos, lo cual nos indica la cantidad de productos que manejan.

El código más recurrente es '85123A'. Por si solo no sabemos si existen muchos productos, solamente que en varias oportunidades, diferentes productos tienen la misma cantidad de productos en un punto dado. Un caso que puede hacer sentido, es que este valor indique una baja cantidad de productos, ya que muchos productos pasan por este valor. Se puede notar que el gráfico va disminuyendo, lo que puede indicar que al extremo izquierdo existan productos con poco stock, ya que no existen celdas vacias. 

In [ ]:
profile_serie(df_retail['Description']) 


La variable "Description" es de tipo categórico y contiene un total de 525.461 observaciones. En esta variable se encuentran 4.682 valores únicos, lo cual indica la cantidad de productos diferentes presentes en el dataset.

La descripción del producto más comúnmente registrado en las diferentes facturas es 'WHITE HANGING HEART T-LIGHT HOLDER'. Aunque esto no necesariamente significa que sea el producto más vendido en términos de cantidad, puede indicar que es un producto necesario y de compra frecuente. Esta descripción de producto aparece en 3.549 facturas.

In [ ]:
profile_serie(df_retail['Quantity']) 

La variable "Quantity" es de tipo entero y contiene 525.461 observaciones en total.

La media es de 10,34 y la desviación estándar es de 107,42. De nuevo se puede observar a través de los gráficos que la media está fuera de los cuantiles 1 y 3, lo cual es un indicador de que hay valores muy grandes que provocan, como el máximo que es 19152, que la media y la desviación se vean afectadas. El tener una gran desviación estándar suele indicar gran variabilidad, en este caso debido en gran parte a los outliers es que existe esta gran variabilidad, mientras que en el gráfico se puede observar una gran concentracion alrededor de la mediana. 

Que el valor mínimo sea un valor negativo, se debe considerar un error, ya que las cantidades no se deben colocarcomo valores negativos, ya que no tiene sentido tener una cantidad negativas de cosas. Si fue colocado a conciencia, puede significar devoluciones, pero aun así no es recomendable utilizar esa medida, en lugar de utilizar una columna nueva, ya que estos valores se utlizan para calcular la media y la desviación. Existen 12.326 observaciones con cantidades negativas.


No se encuentran celdas con cantidad igual a cero en la variable "Quantity".

El coeficiente de curtosis (Kurtosis) tiene un valor de aproximadamente 6.277,67, lo que indica una distribución leptocúrtica con colas pesadas.

El coeficiente de asimetría (Skewness) es positivo y tiene un valor de aproximadamente 36,04, lo que indica una asimetría positiva significativa en la distribución de las cantidades.

No se encuentran celdas faltantes en la variable "Quantity". Todos los registros tienen un valor válido.

Sin embargo, se identifican 525.122 filas duplicadas en la variable "Quantity", lo que representa el 99.94% de todas las observaciones. Esto es normal en el contexto de las compras, ya que todos los productos se pueden comprar en las cantidades que el cliente desee.

In [ ]:
profile_serie(df_retail['InvoiceDate']) 

La variable "InvoiceDate" es de tipo datetime y contiene 525.461 observaciones en total.

La media de la variable es aproximadamente "2010-06-28 11:37:36.845017856", lo cual es muy cercano a la mediana de los resultados que se encuentra en el "2010-06-27"

El valor mínimo de la variable es "2009-12-01 07:45:00", lo que indica la fecha y hora más temprana registrada en el dataset.

Los percentiles muestran que el 25% de las transacciones ocurrieron antes o igual al "2010-03-21 12:20:00", el 50% ocurrieron antes o igual al "2010-07-06 09:51:00", y el 75% ocurrieron antes o igual al "2010-10-15 12:45:00".

El valor máximo de la variable es "2010-12-09 20:01:00", lo que indica la fecha y hora más tardía registrada en el dataset.

No se encuentran celdas faltantes en la variable "InvoiceDate". Todos los registros tienen una fecha y hora válidas.

En el gráfico se puede ver un aumento claro de ventas en un periodo entre finales de septiembre y finales de noviembre. Esto puede ser un buen indicativo a estudiar y comparar con años anteriores para ver si es recurrente este cambio es recurrente o hubo un suceso en particular que desencadenó esa necesidad.

In [ ]:
profile_serie(df_retail['Customer ID']) 

La variable "Customer ID" es de tipo categórico y contiene 417.534 observaciones en total.

Hay 4,383 valores únicos en la variable "Customer ID", lo que indica la presencia de diferentes clientes en el dataset.

El valor más frecuente en la variable "Customer ID" es "14911.0", que aparece 5.710 veces en el dataset. Esta información puede ser muy informativa a la hora del estudio de clientes, ya que al ser un cliente tan frecuente es bueno saber en que contexto vive y de esa forma poder llegar a más personas de estas características o focalizar los esfuerzos.

Hay 107,927 celdas faltantes en la variable "Customer ID", lo que representa aproximadamente el 20.54% de todas las observaciones. Estos registros no tienen un ID de cliente asignado. Esto puede significar errores en el traspaso de los datos. Debido a que las facturas alguien las debe hacer, debe haber un ID del cliente, por lo que no es información que la celdas estén vacias.

Se identifican 525,327 filas duplicadas en la variable "Customer ID", lo que representa el 99.97% de todas las observaciones. Esto sugiere que la mayoría de los IDs de cliente se repiten en el dataset. por lo que quiere decir que solamente el 0.03% compró una única vez.

In [ ]:
profile_serie(df_retail['Country']) 

La variable "Country" es de tipo categórico y contiene 525.461 observaciones en total.

Hay 40 valores únicos en la variable "Country", lo que indica la presencia de 40 países diferentes en el dataset.

El país más frecuente en la variable "Country" es "United Kingdom", que aparece 485,852 veces en el dataset. Esto puede ser muy útil para focalizar las ventas, o entender que existen sucursales muy grandes en "United Kingdom" y se exportan a otros paises u otros casos plausibles.

No se encuentran celdas faltantes en la variable "Country". Todos los registros tienen un valor válido de país.

Se identifican 525,461 filas duplicadas en la variable "Country", lo que representa el 100% de todas las observaciones. Esto indica que todas las filas tienen el mismo país. Esto indica que cada país al menos facturó una vez. 


### 1.2.2 Análisis Multivariado y Datos Faltantes [1 ptos]

En segundo lugar encontrará la función ``profile_df()`` que recibe un dataframe como entrada y realiza un análisis bivariado de todas las variables numéricas que conforman el dataframe, un analisis de la correlación de Pearson entre las variables numericas del dataframe y la matriz de datos faltantes.

In [ ]:
def profile_df(dataframe_in):
    df = dataframe_in.copy()

    list_type = []
    for col in list(df.columns):
        if is_numeric_dtype(df[col]) or \
        pd.core.dtypes.common.is_datetime_or_timedelta_dtype(df[col]):
            list_type.append(col)

    
    display(Markdown('## Bivariant Analysis:'))
    for i in range(len(list_type)):
        for j in range(i+1, len(list_type)):
            plt.scatter(df[list_type[i]], df[list_type[j]])
            plt.xlabel(list_type[i]) 
            plt.ylabel(list_type[j]) 
            plt.title(f"{list_type[i]} v/s {list_type[j]}")
            plt.show()

    display(Markdown('## Correlation:'))
    fig_corr = px.imshow(df[list_type].corr())
    fig_corr.show()

    display(Markdown('## Missing Matrix:'))
    fig, ax = plt.subplots(figsize=[15, 10])
    msno.matrix(df, ax=ax, sparkline=False)

In [ ]:
profile_df(df_retail)

Segun el output del análisis bivariado, se puede oberservar lo siguiente:

- Existen demasiados outliers en general en todas las variables, sin embargo en la variable cantidad existen demasiados outliers con respecto al InvoiceDate en general la cantidad a lo largo de la fecha de invoice esta entre 0 y 1000 (estimación visual) sin embargo algo que se puede notar es que la variable cantidad tiene valores negativos, algo que es imposible, esto podría deberse a valores mal ingresados.
- Respecto a los precios a lo largo del tiempo estos se mantienen en particular estables entre 0 y 5000, sin embargo también existen outliers tanto en precios excesivamente altos como precios negativos los cuales también son imposibles de tener.
- Respecto a la correlación entre variables se observa una correlación muy débil entre la variable cantidad, InvoiceDate y Precio, esto podría deberse a fluctuaciones estacionales en la demanda de productos que afecten las ventas y no se reflejen en la correlación entre cantidad, fecha y precio.
- Por último se observan muchos valores NA en la variable Customer ID, esto puede ser posible a errores humanos en los que no se ingresó esta información.

### 1.2.3 Limpieza de Datos [1 pto]

Como pudo ver en las secciones anteriores, los datos presentan valores erroneos, es por esto que se le solicita que realice una función que permita limpiar el dataset. Realice esta función en base observaciones propias y considere como imposible tener cantidades negativas en las ventas.

Una vez realizada la función, realice nuevamente el análisis exploratorio y comente las principales diferencias.

**Respuesta:**

In [ ]:
def limpiar_df(df):
    # Elimina filas con cantidades negativas
    df = df[df['Quantity'] >= 0]

    # Elimina filas con precios negativos
    df = df[df['Price'] >= 0]

    # Imputación hacia atrás de valores nulos en Customer ID
    df['Customer ID'] = df['Customer ID'].fillna(method='backfill')

    # Tratamiento de outliers
    df = df[df['Quantity'] <= 50]
    df = df[df['Price'] <= 38]

    
    return df


No se realiza una limpieza de valores NA de la variable Customer ID, dado que esta representa el 20% de la cantidad de muestras de esta variable y además podemos estar sesgando nuestra base de datos al eliminar estas filas, en lugar de esto se realiza una imputación backfill de los IDs, al utilizar el método de imputación hacia atrás, se respeta la secuencia temporal de los datos, lo que tiene sentido ya que los datos son basicamente una serie de tiempo de cantidades y precios.

In [ ]:
df_retail  = limpiar_df(df_retail)

In [ ]:
profile_serie(df_retail['Price']) 

Podemos notar que tras la limpieza, se tiene una reducción del número de observaciones, una disminución de los valores extremos y una mayor concentración de los datos alrededor de la media.

In [ ]:
profile_serie(df_retail['Invoice']) 

In [ ]:
profile_serie(df_retail['StockCode']) 

In [ ]:
profile_serie(df_retail['Description']) 

In [ ]:
profile_serie(df_retail['Quantity']) 

Podemos notar que se han eliminado los valores negativos y se ha ajustado la distribución de los datos, sin embargo esto no cambia mucho con respecto a lo que se tenía anteriormente. Además, la limitación del rango máximo para la eliminación de outliers contribuyen a una menor dispersión y consistencia de los datos de cantidades.

In [ ]:
profile_serie(df_retail['InvoiceDate']) 

In [ ]:
profile_serie(df_retail['Customer ID']) 

In [ ]:
profile_serie(df_retail['Country']) 

In [ ]:
profile_df(df_retail)

Podemos notar finalmente, que producto de esta limpieza de valores negativos de cantidad y precio, y el tratamiento de outliers, la correlacion entre InvoiceDate, Price y InvoiceDate, Quantity aumentan, lo que no se podía evidenciar anteriormente, esto sugiere que la limpieza de datos sirvió para dejar la data más consistente para futuros modelos a implementar y reconocer patrones y tendencias más claramente.

### 1.2.4 Obtención de TOPs [0.75 ptos]

Sin considerar los comentarios realizados en la sección 1.2 , *Don Mora* les pide obtener el **Top de 30 productos que generan más ganancias** para la tienda de retail. Deben considerar todo el registro temporal presente en el dataset y entregar la información en un gráfico de barras de los ingresos/cantidades v/s el nombre de los productos (Utilice `plotly`). ¿Los artículos más vendidos son los mismos que generan más ganancias?, Comente los resultados obtenidos.

**Resultados:**

In [ ]:
df_retail['Ganancias'] = df_retail['Price'] * df_retail['Quantity']
top_products_ganancias = df_retail.groupby('Description')['Ganancias'].sum().reset_index()
top_products_ganancias = top_products_ganancias.sort_values(by='Ganancias', ascending=False)
top_products_ganancias = top_products_ganancias.head(30)

fig = px.bar(top_products_ganancias, x='Description', y='Ganancias', title='Top 30 Productos con mayores ganancias')
fig.show()


In [ ]:
top_products_ventas = df_retail.groupby('Description')['Quantity'].sum().reset_index()
top_products_ventas = top_products_ventas.sort_values(by='Quantity', ascending=False)
top_products_ventas = top_products_ventas.head(30)
fig = px.bar(top_products_ventas, x='Description', y='Quantity', title='Top 30 Productos con mayores ventas')
fig.show()

In [ ]:
top_ventas = set(top_products_ventas['Description'])
top_ganancias = set(top_products_ganancias['Description'])

productos_comunes = top_ventas.intersection(top_ganancias)

print("Productos comunes en el top de ventas y top de ganancias:")
for producto in productos_comunes:
    print(producto)


Podemos notar que si bien hay productos que se venden más y generan a su vez mas ganancias para el retail, no todos los productos que estan en el Top 30 de mas vendidos son los Top 30 que generan mas ganancias, esto se puede deber a que puede haber productos que se vendan menos pero que cuesten mas caros o lo contrario que se cuesten muy barato y por tanto se vendan mas a tal pero que las ganancias no sean suficientes, esto ya depende del tipo de productos que tiene el retail que como podemos ver son variados y podrian estar vendiendose harto y generando mayores ganancias debido a que tienen algun diferenciador con los demas productos de su respectiva categoría o simplemente por una mayor estacionalidad lo que aumenta las ventas i.e ingresos respecto a estos productos.

### 1.2.5 Visualización del registro temporal [0,75 ptos]

El dueño del retail en su afán por saber más sobre los datos de su firma les solicita que grafiquen las ventas respecto al tiempo. Con esto les aclara que durante el día tienen muchas variaciones en sus ventas, por lo que les recomienda que consideren el registro temporal como `año-mes-día`. ¿Es posible observar datos extraños?, Comente lo que observa del gráfico.

In [ ]:
def plot_ventas(dataframe):
    dataframe['Fecha'] = dataframe['InvoiceDate'].dt.strftime('%Y-%m-%d')
    ventas_diarias = dataframe.groupby('Fecha')['Quantity'].sum().reset_index()
    fig = px.line(ventas_diarias, x='Fecha', y='Quantity', title='Ventas Diarias')
    fig.show()


In [ ]:
plot_ventas(df_retail)

En efecto, es posible observar una alta variabilidad de las ventas en función de los días, se puede notar un aumento de las ventas a partir de Septiembre esto podría deberse a caracteristicas como la estación del año, vacaciones de escolares/universitarios, etc. Sin embargo este aumento explosivo cambia radicalmente a una disminución drástica en los días previos a navidad que se extiende hasta la primera semana de Enero, esta reducción drástica debe ser un aspecto a considerar por el dueño del retail, ya que no puede ser que se venda menos que en otros meses en los dias de navidad y año nuevo donde se supone hay más demanda, esto podría deberse a cuestiones particulares de la tienda como un quiebre de stock, cierre de la tienda en esas fechas u otro problema puntual a solucionar por el dueño. Finalmente, a partir de la segunda semana de Enero y de nuevo hasta Septiembre la demanda se estabiliza, es recomendable que el dueño le preste atención a estas anomalías y variaciones bruscas para así aumentar sus ventas y que al retail le vaya bien.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>